CNN model and optimization

In [1]:
!pip install keras-tuner


     |████████████████████████████████| 71kB 3.3MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78935 sha256=043f59c4f0409af7248867d7a031d03aefa2a849ed5fac8f8f1af6355aefe680
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=04a453e1cfe86a5aefbde4a9a61a3016ad6dcf2db0aaa97594a37c910ce65ca4
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np 
import pandas as pd


In [3]:
fashion_mnist= keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
#  All the images are in the grey scale. so Now scaling the images into 0 to 1 so that it will help us to quickly train our model

In [6]:
train_images= train_images/255.0
test_images= test_images/255.0

In [7]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 45s]
val_accuracy: 0.8654999732971191

Best val_accuracy So Far: 0.9023333191871643
Total elapsed time: 00h 04m 19s
INFO:tensorflow:Oracle triggered exit


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1996 - accuracy: 0.9254 - val_loss: 0.2674 - val_accuracy: 0.9065
Epoch 5/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1674 - accuracy: 0.9377 - val_loss: 0.2729 - val_accuracy: 0.9058
Epoch 6/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.1418 - accuracy: 0.9470 - val_loss: 0.2958 - val_accuracy: 0.9092
Epoch 7/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.1198 - accuracy: 0.9560 - val_loss: 0.2952 - val_accuracy: 0.9105
Epoch 8/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0983 - accuracy: 0.9633 - val_loss: 0.3704 - val_accuracy: 0.8958
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0834 - accuracy: 0.9689 - val_loss: 0.3588 - val_accuracy: 0.9127
Epoch 10/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0749 - accuracy: 0.9715 - val_loss: 0.4019 - val_a